# **Diseño de compensador adelanto - atraso**

## 1. **Descripcion del sistema**

Sistema de segundo orden con la función de transferencia:
$$ G(s) = \frac{2.671 \times 10^5}{s^2 + 13.07s + 130.5} $$
Con parámetros:
Ganancia: $ K = 2047.0508 $

Frecuencia natural: $ \omega_n = 11.4227 \, \text{rad/s} $ 

Coeficiente de amortiguamiento: $ \zeta = 0.5723 $

Esto corresponde a la forma estándar:
$$ G(s) = \frac{K \omega_n^2}{s^2 + 2 \zeta \omega_n s + \omega_n^2} $$

## **2. Objetivos de Diseño**


**Aumentar el Margen de Fase a 50°:** Esta es una medida de estabilidad. Un margen de fase bajo significa que el sistema está cerca de volverse inestable (oscilar sin control). Aumentarlo a 50° lo hace mucho más robusto y seguro.

**Reducir el Error en Estado Estacionario:** Esto se refiere a la precisión. Si le pides al sistema que llegue a una posición, el error es la diferencia entre dónde querías que llegara y dónde realmente se quedó. Reducir este error lo hace más preciso.

Para lograr esto, se usará un diseño en dos partes:

    1. Compensador de Adelanto: Para mejorar la estabilidad.

    2. Compensador de Atraso: Para mejorar la precisión.

## **3. Diseño de compensador en adelanto - atraso ($ K_c $, $ z_c $, $ p_c $, $ z_a $, $ p_a $)**

### **Asunciones Clave en el Código**

1. **Los Objetivos de Diseño**

    El código asume un margen de fase de 50° y un factor de mejora de error de 10.

    Si el sistema requiere más estabilidad (ej. 60°) o mayor precisión (ej. un factor de 20), se debe cambiar estos valores al principio del código:

In [1]:
pm_deseado = 60      # Nuevo margen de fase
mejora_error = 20     # Nuevo factor de mejora de error

2. **El Periodo de Muestreo (T)**

    Se eligió un periodo de muestreo T = 0.01 segundos.

    Para una correcta implementación, el muestreo debe ser lo suficientemente rápido. Una regla general es que la frecuencia de muestreo ($f_s = \frac{1}{T}$) sea entre 10 y 20 veces más alta que la frecuencia de cruce final ($wc_{final}$).

3. **Calculo de la Ganancia $K_c$**

    El objetivo principal del diseño de adelanto es lograr un margen de fase específico en una nueva frecuencia de cruce de ganancia (w_c,nueva). Por definición, en esta frecuencia, la magnitud (la ganancia) total del sistema compensado debe ser exactamente 1 (o, lo que es lo mismo, 0 decibelios (dB)).

    El cálculo se hace en 3 pasos:

    **3.1. Diseñar los Compensadores sin Ganancia:** Primero, diseñamos las partes de adelanto (C_lead) y atraso (C_lag) que se encargan de arreglar la fase y el error, respectivamente. En este punto, ignoramos la ganancia general.

    **3.2 Medir el Desajuste de Ganancia:** El script calcula cuál es la ganancia del "sistema original + compensador sin K_c" justo en la frecuencia clave w_c,nueva.

    Esta línea mide la magnitud (ganancia) total sin Kc en wc_nueva:

    **mag_total_sin_kc, _, _ = ct.bode(G * C_sin_Kc, [wc_nueva], plot=False)**

    Este valor mag_total_sin_kc casi nunca será 1. Será un número como 1.15, 0.8, etc. Este es el "desajuste" que debemos corregir.

    **3.3 Calcular el Ajuste Perfecto:** K_c es simplemente el número que se necesita para corregir ese desajuste. Si la ganancia actual es mag_total_sin_kc, queremos que:

    $$K_c \times mag\_ total\_ sin\_ K_c = 1$$

    Despejando $K_c$, se obtiene la fórmula usada en el código:

    $$K_c = \frac{1}{mag\_ total\_ sin\_ K_c}$$


### **Código de Automatización**

In [2]:
import numpy as np
import control as ct
import math
import matplotlib.pyplot as plt

# --- 1. Definición del Sistema y Objetivos ---

# Función de transferencia del sistema original (la planta)
num_g = [2.671e5]
den_g = [1, 13.07, 130.5]
G = ct.tf(num_g, den_g)

# Objetivos de diseño
pm_deseado = 50      # Margen de fase deseado en grados
margen_seguridad = 5 # Margen de seguridad en grados
mejora_error = 10     # Factor de mejora para el error estacionario (beta)
T = 0.1             # Periodo de muestreo para la discretización

print("Sistema Original G(s):")
print(G)

# --- 2. Análisis del Sistema Original ---

# Obtenemos el margen de ganancia y fase del sistema sin compensar
gm, pm_actual, wg, wc_actual = ct.margin(G)
print(f"\nMargen de Fase Actual: {pm_actual:.2f}° en la frecuencia {wc_actual:.2f} rad/s")


# --- 3. Diseño del Compensador de Adelanto (Lead) ---
# Esta parte mejora la estabilidad (margen de fase).

# Fase adicional requerida
phi_m = pm_deseado - pm_actual + margen_seguridad
phi_m_rad = math.radians(phi_m)

# Calcular el parámetro 'alpha' para el compensador
alpha = (1 - math.sin(phi_m_rad)) / (1 + math.sin(phi_m_rad))

# El compensador de adelanto añade una ganancia. La nueva frecuencia de cruce (wc_nueva)
# estará donde la magnitud del sistema original sea -10 * log10(1/alpha) dB.
ganancia_lead_db = -10 * np.log10(1 / alpha)

# Encontramos la nueva frecuencia de cruce de ganancia
# Se crea un vector de frecuencias para buscar
frecuencias = np.logspace(0, 3, 1000)
mag_g, _, _ = ct.bode(G, frecuencias, plot=False)
mag_g_db = 20 * np.log10(mag_g)

# Buscamos la frecuencia donde la magnitud coincide con la ganancia necesaria
idx = np.argmin(np.abs(mag_g_db - ganancia_lead_db))
wc_nueva = frecuencias[idx]

# Calculamos el cero (zc) y el polo (pc) del compensador de adelanto
zc = wc_nueva * math.sqrt(alpha)
pc = wc_nueva / math.sqrt(alpha)

# Creamos el compensador de adelanto (aún sin Kc)
C_lead = ct.tf([1, zc], [1, pc])


# --- 4. Diseño del Compensador de Atraso (Lag) ---
# Esta parte mejora la precisión (reduce el error estacionario).

# Colocamos el cero (za) una década por debajo de la nueva frecuencia de cruce
# para minimizar su impacto en la fase.
za = wc_nueva / 10

# El polo (pa) se calcula con el factor de mejora del error (beta)
pa = za / mejora_error

# Creamos el compensador de atraso
C_lag = ct.tf([1, za], [1, pa])


# --- 5. Combinación y Cálculo de Kc ---

# Combinamos ambos compensadores
C_sin_Kc = C_lead * C_lag

# Calculamos la ganancia Kc para que la magnitud total en wc_nueva sea 1 (0 dB)
mag_total_sin_kc, _, _ = ct.bode(G * C_sin_Kc, [wc_nueva], plot=False)
Kc = 1 / mag_total_sin_kc[0]


# --- 6. Resultados Finales ---

print("\n--- Resultados del Diseño ---")
print(f"Frecuencia de cruce objetivo: {wc_nueva:.2f} rad/s")
print(f"Fase requerida: {phi_m:.2f}°")

print("\nParámetros del Compensador Calculados:")
print(f"Kc = {Kc:.4f}")
print(f"zc (cero de adelanto) = {zc:.4f}")
print(f"pc (polo de adelanto) = {pc:.4f}")
print(f"za (cero de atraso) = {za:.4f}")
print(f"pa (polo de atraso) = {pa:.4f}")

print("\nCompensador Continuo Final C(s):")
print(C_s)


Sistema Original G(s):
<TransferFunction>: sys[0]
Inputs (1): ['u[0]']
Outputs (1): ['y[0]']


      2.671e+05
---------------------
s^2 + 13.07 s + 130.5


Margen de Fase Actual: 1.45° en la frecuencia 516.86 rad/s

--- Resultados del Diseño ---
Frecuencia de cruce objetivo: 901.48 rad/s
Fase requerida: 53.55°

Parámetros del Compensador Calculados:
Kc = 9.1946
zc (cero de adelanto) = 296.8202
pc (polo de adelanto) = 2737.8931
za (cero de atraso) = 90.1478
pa (polo de atraso) = 9.0148

Compensador Continuo Final C(s):


C:\Users\Miguel\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\control\freqplot.py:435: FutureWarning: bode_plot() return value of mag, phase, omega is deprecated; use frequency_response()
  warnings.warn(


NameError: name 'C_s' is not defined